In [1]:
from glob import glob
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from fastai.layers import SigmoidRange

In [2]:
def rmspe_metric(y_true, y_pred):

    """
    Calculate root mean squared percentage error between ground-truth and predictions

    Parameters
    ----------
    y_true [array-like of shape (n_samples)]: Ground-truth
    y_pred [array-like of shape (n_samples)]: Predictions

    Returns
    -------
    rmspe (float): Root mean squared percentage error
    """

    rmspe = np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))
    return rmspe


def evaluate_predictions(predictions_column, evaluate_stock=False):
    
    """
    Evaluate predictions of the given model for every individual stock and entire training set

    Parameters
    ----------
    df_train [pandas.DataFrame of shape (n_samples)]: DataFrame with stock_id and prediction columns
    predictions_column (str): Predictions of the model
    """
    
    for fold in sorted(df_train['fold'].unique()):

        _, val_idx = df_train.loc[df_train['fold'] != fold].index, df_train.loc[df_train['fold'] == fold].index
        fold_score = rmspe_metric(df_train.loc[val_idx, 'target'], df_train.loc[val_idx, predictions_column])
        print(f'Fold {fold} - RMSPE: {fold_score:.6}')

    oof_score = rmspe_metric(df_train['target'], df_train[predictions_column])
    print(f'{"-" * 30}\nOOF RMSPE: {oof_score:.6}\n{"-" * 30}')

    if evaluate_stock:
        for stock_id in df_train['stock_id'].unique():
            df_stock = df_train.loc[df_train['stock_id'] == stock_id, :]
            stock_oof_score = rmspe_metric(df_stock['target'], df_stock[predictions_column])
            print(f'Stock {stock_id} - OOF RMSPE: {stock_oof_score:.6}')


In [3]:
class PreprocessingPipeline:

    def __init__(self, df_train, df_test, split_type):

        self.df_train = df_train.copy(deep=True)
        self.df_test = df_test.copy(deep=True)
        self.split_type = split_type

    def _label_encode(self):

        le = LabelEncoder()
        self.df_train['stock_id_encoded'] = le.fit_transform(self.df_train['stock_id'].values)
        self.df_test['stock_id_encoded'] = le.transform(self.df_test['stock_id'].values)

    def _get_folds(self):

        df_folds = pd.read_csv('../input/optiver-realized-volatility-dataset/folds.csv')
        self.df_train['fold'] = df_folds[f'fold_{self.split_type}']
        
    def transform(self):

        self._get_folds()
        self._label_encode()

        return self.df_train, self.df_test


In [4]:
train_test_dtypes = {
    'stock_id': np.uint8,
    'time_id': np.uint16,
    'target': np.float64
}

df_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv', dtype=train_test_dtypes)
df_test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv', usecols=['stock_id', 'time_id'], dtype=train_test_dtypes)

# Using only first row of test set if it is the placeholder
# Other rows break the pipeline since some of the time buckets don't exist in book data
if df_test.shape[0] == 3:
    df_test = df_test.head(1)

preprocessing_pipeline = PreprocessingPipeline(df_train, df_test, 'group')
df_train, df_test = preprocessing_pipeline.transform()

print(f'Training Set Shape: {df_train.shape} - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Test Set Shape: {df_test.shape} - Memory Usage: {df_test.memory_usage().sum() / 1024 ** 2:.2f} MB')

Training Set Shape: (428932, 5) - Memory Usage: 11.04 MB
Test Set Shape: (1, 3) - Memory Usage: 0.00 MB


In [5]:
class MLPBlock(nn.Module):

    def __init__(self, input_dim, hidden_dim, dropout_rate):

        super(MLPBlock, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim, bias=True),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, input_dim, bias=True),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        return self.mlp(x)


class MixerBlock(nn.Module):

    def __init__(self, num_patches, hidden_dim, token_mixer_dim, token_mixer_dropout_rate, channel_mixer_dim, channel_mixer_dropout_rate):

        super(MixerBlock, self).__init__()

        self.layer_norm_token = nn.LayerNorm(hidden_dim)
        self.token_mixer = MLPBlock(num_patches, token_mixer_dim, token_mixer_dropout_rate)
        self.layer_norm_channel = nn.LayerNorm(hidden_dim)
        self.channel_mixer = MLPBlock(hidden_dim, channel_mixer_dim, channel_mixer_dropout_rate)

    def forward(self, x):

        out = self.layer_norm_token(x).transpose(1, 2)
        x = x + self.token_mixer(out).transpose(1, 2)
        out = self.layer_norm_channel(x)
        x = x + self.channel_mixer(out)

        return x


class MLPMixerModel(nn.Module):

    def __init__(self, sequence_length, channels, patch_size, hidden_dim, num_blocks, token_mixer_dim, token_mixer_dropout_rate, channel_mixer_dim, channel_mixer_dropout_rate, use_stock_id, stock_embedding_dims):

        super(MLPMixerModel, self).__init__()

        # Stock embeddings
        self.use_stock_id = use_stock_id
        self.stock_embedding_dims = stock_embedding_dims
        self.stock_embeddings = nn.Embedding(num_embeddings=113, embedding_dim=self.stock_embedding_dims)
        self.dropout = nn.Dropout(0.25)

        # Patch embeddings
        num_patches = (sequence_length // patch_size[0]) * (channels // patch_size[1])
        self.patch_embeddings = nn.Conv2d(1, hidden_dim, kernel_size=patch_size, stride=patch_size, bias=False)

        # Mixer blocks
        self.mixer = nn.Sequential(
            *[
                MixerBlock(
                    num_patches=num_patches,
                    hidden_dim=hidden_dim,
                    token_mixer_dim=token_mixer_dim,
                    token_mixer_dropout_rate=token_mixer_dropout_rate,
                    channel_mixer_dim=channel_mixer_dim,
                    channel_mixer_dropout_rate=channel_mixer_dropout_rate
                ) for _ in range(num_blocks)
            ]
        )

        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.head = nn.Sequential(
            nn.Linear(hidden_dim + self.stock_embedding_dims, 1, bias=True),
            SigmoidRange(0, 0.1)
        )

    def forward(self, stock_ids, sequences):

        x = sequences.view(-1, 1, sequences.shape[1], sequences.shape[2])
        x = self.patch_embeddings(x)
        x = x.flatten(2).transpose(1, 2)
        x = self.mixer(x)
        x = self.layer_norm(x)
        x = x.mean(dim=1)

        if self.use_stock_id:
            embedded_stock_ids = self.stock_embeddings(stock_ids)
            x = torch.cat([x, self.dropout(embedded_stock_ids)], dim=1)

        output = self.head(x)
        return output.view(-1)


In [6]:
# Normalizing sequences with global means and stds
book_means = np.array([
    # Raw sequences
    0.99969482421875, 1.000321388244629, 0.9995064735412598, 1.0005191564559937,
    769.990177708821, 766.7345672818379, 959.3416027831918, 928.2202512713748,
    # Absolute log returns of raw sequences
    5.05890857311897e-05, 5.1026330766035244e-05, 5.74059049540665e-05, 5.8218309277435765e-05,
    0.3967152245253066, 0.39100519899866804, 0.3239659116907835, 0.31638538484106116,
    # Weighted average prices
    1.0000068043192514, 1.0000055320253616, 1.000006872969592,
    # Absolute log returns of weighted average prices
    8.211420490291096e-05, 0.00011112522790786203, 8.236187150264073e-05
])
book_stds = np.array([
    # Raw sequences
    0.0036880988627672195, 0.003687119111418724, 0.0037009266670793295, 0.0036990800872445107,
    5354.051690318169, 4954.947103063445, 6683.816183660414, 5735.299917793827,
    # Absolute log returns of raw sequences
    0.00016576898633502424, 0.00016801751917228103, 0.0001837657910073176, 0.0001868011022452265,
    0.9121719707304721, 0.8988021131995019, 0.8415323589617927, 0.8244750862945265,
    # Weighted average prices
    0.003689893218043926, 0.00370745215558702, 0.0036913980961173682,
    # Absolute log returns of weighted average prices
    0.00021108155612872302, 0.00029320157822289604, 0.00019975085953727163
])
trade_means = np.array([0.999971866607666, 352.9736760331942, 4.1732040971227145])
trade_stds = np.array([0.004607073962688446, 1041.9441951057488, 7.79955795393431])

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

mlp_mixer_parameters = {
    'sequence_length': 600,
    'channels': 25,
    'patch_size': (30, 1),
    'hidden_dim': 128,
    'num_blocks': 2,
    'token_mixer_dim': 64,
    'token_mixer_dropout_rate': 0,
    'channel_mixer_dim': 64,
    'channel_mixer_dropout_rate': 0,
    'use_stock_id': True,
    'stock_embedding_dims': 16
}

print(f'MLP Mixer Models\n{"-" * 16}')
mlp_mixer_models = []
for model_path in sorted(glob(f'../input/optiver-realized-volatility-dataset/mlp_mixer/*.pt')):
    print(f'Loading model {model_path} into memory')
    model = MLPMixerModel(**mlp_mixer_parameters)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    mlp_mixer_models.append(model)
    

MLP Mixer Models
----------------
Loading model ../input/optiver-realized-volatility-dataset/mlp_mixer/mlp_mixer_fold1.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/mlp_mixer/mlp_mixer_fold2.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/mlp_mixer/mlp_mixer_fold3.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/mlp_mixer/mlp_mixer_fold4.pt into memory
Loading model ../input/optiver-realized-volatility-dataset/mlp_mixer/mlp_mixer_fold5.pt into memory


In [7]:
# Loading pre-computed train predictions and evaluate them
df_train['mlp_mixer_predictions'] = pd.read_csv('../input/optiver-realized-volatility-dataset/mlp_mixer/mlp_mixer_predictions.csv').values

print(f'MLP Mixer Scores\n{"-" * 16}')
evaluate_predictions('mlp_mixer_predictions', evaluate_stock=True)

MLP Mixer Scores
----------------
Fold 1 - RMSPE: 0.241555
Fold 2 - RMSPE: 0.24438
Fold 3 - RMSPE: 0.231773
Fold 4 - RMSPE: 0.230869
Fold 5 - RMSPE: 0.228275
------------------------------
OOF RMSPE: 0.235457
------------------------------
Stock 0 - OOF RMSPE: 0.273276
Stock 1 - OOF RMSPE: 0.210029
Stock 2 - OOF RMSPE: 0.209807
Stock 3 - OOF RMSPE: 0.229543
Stock 4 - OOF RMSPE: 0.263184
Stock 5 - OOF RMSPE: 0.261158
Stock 6 - OOF RMSPE: 0.213594
Stock 7 - OOF RMSPE: 0.246875
Stock 8 - OOF RMSPE: 0.21915
Stock 9 - OOF RMSPE: 0.264
Stock 10 - OOF RMSPE: 0.190834
Stock 11 - OOF RMSPE: 0.238488
Stock 13 - OOF RMSPE: 0.221163
Stock 14 - OOF RMSPE: 0.189802
Stock 15 - OOF RMSPE: 0.210465
Stock 16 - OOF RMSPE: 0.280504
Stock 17 - OOF RMSPE: 0.207725
Stock 18 - OOF RMSPE: 0.306713
Stock 19 - OOF RMSPE: 0.245198
Stock 20 - OOF RMSPE: 0.191604
Stock 21 - OOF RMSPE: 0.247961
Stock 22 - OOF RMSPE: 0.224264
Stock 23 - OOF RMSPE: 0.217327
Stock 26 - OOF RMSPE: 0.219135
Stock 27 - OOF RMSPE: 0.285772

In [8]:
book_features = ['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2','bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']
trade_features = ['price', 'size', 'order_count']
stock_id_mapping = df_train.set_index('stock_id')['stock_id_encoded'].to_dict()

for stock_id in tqdm(df_test['stock_id'].unique()):
    
    df_stock = df_test.loc[df_test['stock_id'] == stock_id]
    df_book = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id={stock_id}')
    df_trade = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id={stock_id}')
    stock_time_buckets = df_test.loc[df_test['stock_id'] == stock_id, 'time_id'].reset_index(drop=True)
    missing_time_buckets = stock_time_buckets[~stock_time_buckets.isin(df_trade['time_id'])]
    df_trade = df_trade.merge(missing_time_buckets, how='outer')
    
    # Iterating over time_ids
    for time_id in df_stock['time_id'].unique():
        
        # Resample order book to 600 seconds, forward fill and back fill for edge cases
        df_book_time_bucket = df_book.loc[df_book['time_id'] == time_id]
        df_book_time_bucket = df_book_time_bucket.set_index(['seconds_in_bucket'])
        df_book_time_bucket = df_book_time_bucket.reindex(np.arange(0, 600), method='ffill').fillna(method='bfill')
        
        # Sequences from book data
        book_sequences = df_book_time_bucket.reset_index(drop=True)[book_features].values
        
        # Absolute log returns of raw sequences
        book_bid_price1_log = np.log(book_sequences[:, 0])
        book_bid_price1_absolute_log_returns = np.abs(np.diff(book_bid_price1_log, prepend=[book_bid_price1_log[0]]))
        book_ask_price1_log = np.log(book_sequences[:, 1])
        book_ask_price1_absolute_log_returns = np.abs(np.diff(book_ask_price1_log, prepend=[book_ask_price1_log[0]]))
        book_bid_price2_log = np.log(book_sequences[:, 2])
        book_bid_price2_absolute_log_returns = np.abs(np.diff(book_bid_price2_log, prepend=[book_bid_price2_log[0]]))
        book_ask_price2_log = np.log(book_sequences[:, 3])
        book_ask_price2_absolute_log_returns = np.abs(np.diff(book_ask_price2_log, prepend=[book_ask_price2_log[0]]))
        book_bid_size1_log = np.log(book_sequences[:, 4])
        book_bid_size1_absolute_log_returns = np.abs(np.diff(book_bid_size1_log, prepend=[book_bid_size1_log[0]]))
        book_ask_size1_log = np.log(book_sequences[:, 5])
        book_ask_size1_absolute_log_returns = np.abs(np.diff(book_ask_size1_log, prepend=[book_ask_size1_log[0]]))
        book_bid_size2_log = np.log(book_sequences[:, 6])
        book_bid_size2_absolute_log_returns = np.abs(np.diff(book_bid_size2_log, prepend=[book_bid_size2_log[0]]))
        book_ask_size2_log = np.log(book_sequences[:, 7])
        book_ask_size2_absolute_log_returns = np.abs(np.diff(book_ask_size2_log, prepend=[book_ask_size2_log[0]]))

        # Weighted average prices
        book_wap1 = (book_sequences[:, 0] * book_sequences[:, 5] + book_sequences[:, 1] * book_sequences[:, 4]) /\
                    (book_sequences[:, 4] + book_sequences[:, 5])
        book_wap2 = (book_sequences[:, 2] * book_sequences[:, 7] + book_sequences[:, 3] * book_sequences[:, 6]) /\
                    (book_sequences[:, 6] + book_sequences[:, 7])
        book_wap3 = ((book_sequences[:, 0] * book_sequences[:, 5] + book_sequences[:, 1] * book_sequences[:, 4]) +
                     (book_sequences[:, 2] * book_sequences[:, 7] + book_sequences[:, 3] * book_sequences[:, 6])) /\
                    (book_sequences[:, 4] + book_sequences[:, 5] + book_sequences[:, 6] + book_sequences[:, 7])

        # Absolute log returns of weighted average prices
        book_wap1_log = np.log(book_wap1)
        book_wap1_absolute_log_returns = np.abs(np.diff(book_wap1_log, prepend=[book_wap1_log[0]]))
        book_wap2_log = np.log(book_wap2)
        book_wap2_absolute_log_returns = np.abs(np.diff(book_wap2_log, prepend=[book_wap2_log[0]]))
        book_wap3_log = np.log(book_wap3)
        book_wap3_absolute_log_returns = np.abs(np.diff(book_wap3_log, prepend=[book_wap3_log[0]]))

        book_sequences = np.hstack([
            book_sequences,
            book_bid_price1_absolute_log_returns.reshape(-1, 1),
            book_ask_price1_absolute_log_returns.reshape(-1, 1),
            book_bid_price2_absolute_log_returns.reshape(-1, 1),
            book_ask_price2_absolute_log_returns.reshape(-1, 1),
            book_bid_size1_absolute_log_returns.reshape(-1, 1),
            book_ask_size1_absolute_log_returns.reshape(-1, 1),
            book_bid_size2_absolute_log_returns.reshape(-1, 1),
            book_ask_size2_absolute_log_returns.reshape(-1, 1),
            book_wap1.reshape(-1, 1),
            book_wap2.reshape(-1, 1),
            book_wap3.reshape(-1, 1),
            book_wap1_absolute_log_returns.reshape(-1, 1),
            book_wap2_absolute_log_returns.reshape(-1, 1),
            book_wap3_absolute_log_returns.reshape(-1, 1),
        ])
        book_sequences = (book_sequences - book_means) / book_stds
        
        # Resample trade data to 600 seconds and fill missing values with 0
        df_trade_time_bucket = df_trade.loc[df_trade['time_id'] == time_id]
        df_trade_time_bucket = df_trade_time_bucket.set_index(['seconds_in_bucket'])
        df_trade_time_bucket = df_trade_time_bucket.reindex(np.arange(0, 600)).fillna(0)
        
        # Sequences from trade data
        trade_sequences = df_trade_time_bucket.reset_index(drop=True)[trade_features].values
        # Not normalizing zero values in trade data
        trade_sequences[trade_sequences[:, 0] != 0, :] = (trade_sequences[trade_sequences[:, 0] != 0, :] - trade_means) / trade_stds
        
        # Concatenate book and trade sequences
        sequences = np.hstack([book_sequences, trade_sequences])
        sequences = torch.as_tensor(sequences.reshape(1, 600, 25), dtype=torch.float)
        sequences = sequences.to(device)
        
        stock_id_encoded = torch.as_tensor([stock_id_mapping[stock_id]], dtype=torch.long)
        stock_id_encoded = stock_id_encoded.to(device)
        
        mlp_mixer_prediction = 0
        for model in mlp_mixer_models:
            with torch.no_grad():
                prediction = model(stock_id_encoded, sequences).detach().cpu().numpy()
                mlp_mixer_prediction += (prediction[0] / 5)
        df_test.loc[(df_test['stock_id'] == stock_id) & (df_test['time_id'] == time_id), 'mlp_mixer_predictions'] = mlp_mixer_prediction


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
df_test['target'] = df_test['mlp_mixer_predictions'].values
df_test['row_id'] = df_test['stock_id'].astype(str) + '-' + df_test['time_id'].astype(str)
df_test[['row_id', 'target']].to_csv('submission.csv', index=False)